# Collision Division - Creating the UAV autonomous navigation system and data pipeline

In [1]:
import time
import csv
import pandas as pd
import math as m

In [2]:
data = pd.read_csv('cleaned_data_collision_division.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)

## Data and values

In [3]:
# icao values
icao_values = list(set(data.icao))

# uav loc df
uav_cols = ['ui', 'timestamp', 'longitude', 'latitude', 'altitude', 'heading']
uav_loc_df = pd.DataFrame(columns=uav_cols)

## Algorithm implementation

### Functions

In [4]:
# Sets the heading of the UAV
def header_warning(uav, plane, uav_turn_angle):
    # uav and plane passed in as array with lat, long, alt, header, km_per_long
    # Finding Quadrants
    x = plane[0] - uav[0]
    y = plane[1] - uav[1]

    # reference header angle for turning
    ref_header = plane[3] - uav[3]

    if (x < 0) & (y > 0): # top left quadrant
        if (ref_header >= 180) & (ref_header <= 360):
            return(uav[3])
        else:
            uav[3] += uav_turn_angle
            uav[3] = uav[3]%360
            return(uav[3])
    elif (x > 0) & (y > 0): # top right quadrant
        if (ref_header >= 0) & (ref_header <= 180):
            return(uav[3])
        else:
            uav[3] -= uav_turn_angle
            uav[3] = uav[3]%360
            return(uav[3])
    elif (x < 0) & (y < 0): # bottom left quadrant
        if (ref_header >= 90) & (ref_header <= 360):
            return(uav[3])
        else:
            uav[3] -= uav_turn_angle
            uav[3] = uav[3]%360
            return(uav[3])
    elif (x > 0) & (y < 0): # bottom right quadrant
        if (ref_header >= 0) & (ref_header <= 270):
            return(uav[3])
        else:
            uav[3] -= uav_turn_angle
            uav[3] = uav[3]%360
            return(uav[3])


In [5]:
# Converts the uav location and error data to a dictionary
def uav_loc_to_df(uav, timestamp, message):
    messages.append(message)
    uav_dict[timestamp] = uav

In [6]:
# Sets the altitude of the UAV
def altitude_warning(uav_altitude, plane_altitude, descent_speed):
    if uav_altitude - plane_altitude < 0:
        uav_altitude -= descent_speed
    else:
        uav_altitude += descent_speed
    return(uav_altitude)

In [7]:
# Converts the coordinates from cartesian format
def convert_to_ellip(input, km_per_lat):
    ellip = {}
    for key in input:
        ## Update cart Dict with Cartesian values
        ellip.update({key:[input[key][0]/input[key][4], input[key][1]/km_per_lat, input[key][2], input[key][3]]})
    return(ellip)


In [8]:
# Converts the coordinates to cartesian format
def convert_to_cart(input, km_per_lat):
    cart = {}
    for key in input:
        ## Longitude speed calculation
        phi = input[key][0] * (m.pi/180)
        km_per_long = km_per_lat * m.cos(phi)

        ## Update cart Dict with Cartesian values
        cart.update({key:[input[key][0]*km_per_long, input[key][1]*km_per_lat, input[key][2], input[key][3], km_per_long]})
    return(cart)

In [9]:
# Finds every location for every plane for the UAV to compare it's location to
def plane_locator(uav_starttime):
    """ Params: uav_starttime - When the UAV starts flying
        Returns: all active planes at that time stamp
    """
    plane_locs = {}
    for icao in icao_values:
        key = icao
        try:
            lat,long,alt,head = data[['latitude', 
                                      'longitude', 
                                      'altitude_(m)', 
                                      'heading']].loc[(data.timestamp==uav_starttime) 
                                                      & (data.icao==icao)].values[0]
        except:
            lat = 0
            long = 0
            alt = 0
            head = 0
            key = None
        plane_locs[key] = [lat, long, alt, head]
    active_plane_locs = {k: v for k, v in plane_locs.items() if k is not None}
    return active_plane_locs

In [10]:
# Defines the UAV speed and heading
def move_distance(heading, speed):
    x = speed * m.cos(heading)
    y = speed * m.sin(heading)
    return(x, y)

In [11]:
# Finds the distance between the UAV and the aircrafts
def find_distance(uav, plane, uav_speed, uav_descent_speed, uav_turn_angle):
    uav_messages = []
    for uav_key in uav:
        for craft_key in plane:
            dist_xy = m.sqrt((plane[craft_key][0] - uav[uav_key][0])**2 + (plane[craft_key][1] - uav[uav_key][1])**2)
            dist_alt = abs(plane[craft_key][2] - uav[uav_key][2])

            if dist_xy <= 10:
                if dist_alt <= 500:
                    if dist_xy <= 5:
                        message = f"Critical! You are {round(dist_xy, 3)} km away!"
                        print(message)
                        uav_messages.append(message)
                    else:
                        message = f"Warning! You are {round(dist_xy, 3)} km away!"
                        print(message)
                        uav_messages.append(message)
                    uav[uav_key][3] = header_warning(uav[uav_key], plane[craft_key], uav_turn_angle)
                    x, y = move_distance(uav[uav_key][3], uav_speed)
                    uav[uav_key][0] += (x/1222)
                    uav[uav_key][1] += (y/1222)
                    uav[uav_key][2] = altitude_warning(uav[uav_key][2], plane[craft_key][2], uav_descent_speed)
                elif dist_alt > 500:
                    x, y = move_distance(uav[uav_key][3], uav_speed)
                    uav[uav_key][0] += (x/1222)
                    uav[uav_key][1] += (y/1222)
            else:
                message = "Clear"
                x, y = move_distance(uav[uav_key][3], uav_speed)
                uav[uav_key][0] += (x/1222)
                uav[uav_key][1] += (y/1222)
                uav_messages.append(message)
    
    for msg in uav_messages:
        if 'Critical!' in msg:
            message = msg
        elif 'Warning!' in msg:
            message = msg

    return(uav, message)

# Main Loop

In [12]:
%%time

# Sets a start location and runs the UAV flight for a set flight time
# Globals
km_per_lat = 111.2403
warning = False
alt_warning = False
uav_speed = 25
uav_descent_speed = 0.002
uav_turn_angle = 10

# uav start position
uav = {'uav_icao_1': [38.879638, 282.379883, 10965.180000, 218.927544]}

# Change this based on when the UAV starts flying
uav_starttime = 1541708062

# how long we want the drone to fly in seconds
flight_time = 240

# Dict that holds all of the UAV values
uav_dict = {}
messages = []
uav_messages_dict = {}

# The main loop (change the while condition based on if UAV has arrived at end location)
i = 0
while i < flight_time:
    timestamp = uav_starttime+i
    print("Timestamp: ", timestamp)
    
    # Required functions
    active_planes = plane_locator(uav_starttime)
    cart = convert_to_cart(active_planes, km_per_lat)
    uav = convert_to_cart(uav, km_per_lat)
    uav, message = find_distance(uav, cart, uav_speed, uav_descent_speed, uav_turn_angle)
    uav = convert_to_ellip(uav, km_per_lat)
    uav_loc_to_df(uav, timestamp, message)
    
    i+=1

Timestamp:  1541708062
Critical! You are 0.02 km away!
Timestamp:  1541708063
Critical! You are 0.065 km away!
Timestamp:  1541708064
Critical! You are 0.071 km away!
Timestamp:  1541708065
Critical! You are 0.055 km away!
Timestamp:  1541708066
Critical! You are 0.176 km away!
Timestamp:  1541708067
Critical! You are 0.298 km away!
Timestamp:  1541708068
Critical! You are 0.421 km away!
Timestamp:  1541708069
Critical! You are 0.544 km away!
Timestamp:  1541708070
Critical! You are 0.666 km away!
Timestamp:  1541708071
Critical! You are 0.789 km away!
Timestamp:  1541708072
Critical! You are 0.912 km away!
Timestamp:  1541708073
Critical! You are 1.034 km away!
Timestamp:  1541708074
Critical! You are 1.157 km away!
Timestamp:  1541708075
Critical! You are 1.28 km away!
Timestamp:  1541708076
Critical! You are 1.403 km away!
Timestamp:  1541708077
Critical! You are 1.525 km away!
Timestamp:  1541708078
Critical! You are 1.648 km away!
Timestamp:  1541708079
Critical! You are 1.771 km 

## Outputting the data

In [13]:
# Converts the uav dicts to a dataframe
timestamps = []
uav_icao = []
latitudes = []
longitudes = []
altitudes = []
headings = []

for k,v in uav_dict.items():
    timestamps.append(k)
    for k1, v1 in v.items():
        uav_icao.append(k1)
        latitudes.append(v1[0])
        longitudes.append(v1[1])
        altitudes.append(v1[2])
        headings.append(v1[3])

d = {'uav_icao': uav_icao, 'timestamp': timestamps,  
    'latitude': latitudes, 'longitudes': longitudes, 
    'altitude': altitudes, 'heading': headings, 
    'message': messages}

uav_df = pd.DataFrame.from_dict(d)

columns = ['uav_icao', 'timestamp', 'latitude', 'longitudes', 'altitude', 'heading', 'message']

uav_df = uav_df.reindex(columns=columns)

In [14]:
# Outputs a csv of the uav data to be exported to the database for plotting
uav_df.to_csv('uav_data.csv')

In [15]:
uav_df

,uav_icao,timestamp,latitude,longitudes,altitude,heading,message
0,uav_icao_1,1541708062,38.878685,282.380095,10965.182,228.927544,Critical! You are 0.02 km away!
1,uav_icao_1,1541708063,38.879122,282.379403,10965.184,218.927544,Critical! You are 0.065 km away!
2,uav_icao_1,1541708064,38.879239,282.380169,10965.186,208.927544,Critical! You are 0.071 km away!
3,uav_icao_1,1541708065,38.879223,282.381272,10965.188,208.927544,Critical! You are 0.055 km away!
4,uav_icao_1,1541708066,38.879206,282.382376,10965.190,208.927544,Critical! You are 0.176 km away!
5,uav_icao_1,1541708067,38.879190,282.383479,10965.192,208.927544,Critical! You are 0.298 km away!
6,uav_icao_1,1541708068,38.879173,282.384583,10965.194,208.927544,Critical! You are 0.421 km away!
7,uav_icao_1,1541708069,38.879157,282.385686,10965.196,208.927544,Critical! You are 0.544 km away!
8,uav_icao_1,1541708070,38.879140,282.386789,10965.198,208.927544,Critical! You are 0.666 km away!
9,uav_icao_1,1541708071,38.879124,282.387893,10965.200,208.927544,Critical! You are 0.789 km away!
